# WeRateDogs - Wrangle and Analyze Data


## Table of Contents
- [Introduction](#intro)
<br><br>
- [Import libaries](#import)
<br><br>
- [Gather data](#gather)
    1. [File on Hand](#gather_1)
    2. [File from Internet](#gather_2)
    3. [API data](#gather_3)
<br><br>
- [Assess Data](#assess)
    1. [File on Hand: df_twitter_archive](#assess_1)
    2. [File from Internet: df_images](#assess_2)
    3. [API data: df_tweets](#assess_3)
    4. [Quality Issues](#assess_qual)
    5. [Tidiness Issues](#assess_tidy)
<br><br>
- [Clean Data](#clean)
    1. [Missing Data](#clean_miss)
    2. [Tidiness Issues](#clean_tidy)
    3. [Quality Issues](#clean_qual)
<br><br>
- [Save Cleaned Datasets](#store)
<br><br>
- [Analyze and Visualize](#analyze)
<br><br>
- [Conclusions](#conclusion)

    
    
<a id='intro'></a>
## Introduction

*WeRateDogs* is a Twitter account that gives humourous comments and ratings of people's dogs.  It has over 8 million followers and international media coverage.

In order to find interesting insights about whether and how the Dog Rating Tweets relate to the popularity of *WeRateDogs* Twitter account, data must first be gathered, assessed and clean.  Three datasets are available for this project:
   1. File on hand: WeRateDogs Twitter archive, which includes basic tweet data for over 2,000 tweets that have ratings
   2. File from the internet: Image Prediction file that gives the top 3 predictions of a dog's breed based on a jpeg image associated with each tweet that has been run through a neural network.  This file is hosted on Udacity's server
   3. Twitter API data: query Twitter's API in order to gather additional and valuable information.  Save the JSON data to text
    
The data is then assessed both visually and programatically to find any quality or tidiness issues.  Quality issues relate to the data content and Tidiness issues relate to the structure of the data.  Each issue is then in turn treated and cleaned.

Finally the data will be explored, guided by questions about how the Dog Rating Tweets relate to the phenomenal success and popularity of *WeRateDogs* Twitter account.

#### *Key Points*
- We only want original tweets (no retweets) that have images
- The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of *WeRateDogs*.
- Tweets do not extend beyond August 1, 2017



<a id='import'></a>
## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import requests
import os
import tweepy
import wptools

import json
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<a id='gather'></a>
## Gather

<a id='gather_1'></a>
### *Gather 1. File on Hand*

WeRateDogs Twitter archive, `twitter-archive-enhanced.csv`


In [ ]:
# Read in file on hand twitter-archive-enhanced.csv
df_twitter_archive = pd.read_csv('./data/twitter-archive-enhanced.csv')
df_twitter_archive.head()

<a id='gather_2'></a>
### *Gather 2. File from internet*

Image Prediction file `image_predictions.tsv`is hosted on Udacity's server at  https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [ ]:
# Check for and/or Create folder for data
folder_name = 'data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
#Download image Tweet image prediction file 
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response

In [ ]:
# Write content to local computer
with open(os.path.join(folder_name,
                      url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

In [ ]:
# Check encoding on downloaded file
with open('{}/image-predictions.tsv'.format(folder_name)) as file:
    print(file)

In [ ]:
# Read image prediction data into DataFrame df_images
df_images = pd.read_csv('{}/image-predictions.tsv'.format(folder_name), sep='\t')
df_images.head()

<a id='gather_3'></a>
### *Gather 3. Twitter's API data*
Query Twitter's API using Tweepy and save JSON in a text file `tweet_json.txt`

In [ ]:
# Import hidden Twitter API keys, secrets and tokens
import creds
auth = tweepy.OAuthHandler(creds.consumer_key, creds.consumer_secret)
auth.set_access_token(creds.access_token, creds.access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# Create tweet_id list based on tweet_id in Twitter archive df_twitter_archive
tweet_id_list = df_twitter_archive['tweet_id'].unique()

In [ ]:
# Query Twitter's API for JSON data for each tweet_id in tweet_id_list
tweet_data=[]
tweet_error={}

tweet_count=0
error_count=0

start=timer()

for tweet_id in tweet_id_list:
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        tweet_data.append(tweet._json)
        tweet_count +=1
        print(tweet.id)
        
    except Exception as e:
        tweet_error[str(tweet_id)]=str(e)
        error_count +=1
        print(str(tweet_id) + ": " + str(e))
        
end=timer()
print('Elapsed time: {}'.format(end-start))
print('Tweet count: {}'.format(tweet_count))
print('Error count: {}'.format(error_count))

In [ ]:
tweet_error

In [ ]:
# Manually get data for tweet_ids if error is other than 'No status found with that ID'
for key in tweet_error.keys():
    if not ("'code': 144" in tweet_error.get(key)):
        tweet = api.get_status(key, tweet_mode='extended')
        tweet_data.append(tweet._json)
        print(key)
    

In [ ]:
# Save Tweet JSON data to tweet_json.txt
with open('{}/tweet_json.txt'.format(folder_name), 'w') as outfile:
    for json_tweet in tweet_data:
        json.dump(json_tweet, outfile)
        outfile.write('\n')

In [ ]:
# Read json_tweet.txt line by line to extract tweet_id, retweet_count and favourite_count
json_tweet_data = []
with open('{}/tweet_json.txt'.format(folder_name)) as f:
    for line in f:
        json_data = json.loads(line)
        tweet_id = json_data['id']
        retweet_count = json_data['retweet_count']
        favourite_count = json_data['favorite_count']
        json_tweet_data.append({'tweet_id': tweet_id,
                                'retweet_count': retweet_count,
                               'favourite_count': favourite_count})

In [ ]:
df_tweets = pd.DataFrame(json_tweet_data, columns=['tweet_id','retweet_count', 'favourite_count'])
df_tweets.head()

<a id='assess'></a>
## Assess

Assess the data first visually then programatically for Quality and Tidiness issues.

***Quality*** issues relate to content.  The four main quality dimensions are:
- *Completeness* : do we have all the records that we should and are the specific rows, columns or cells missing?
- *Validitiy* : does the date conform to the defined schema, or set of rules?
- *Accuracy* : data that may conform to the schema but is still incorrect
- *Consistency* : is data presented consistently within and across tables

***Tidiness*** issues relate to structure. The basic principals of tidy data are:
- Each variable forms a column
- Each observation forms a row
- Each type of observational unit forms a table


<a id='assess_1'></a>
### *Assess 1. File on Hand: df_twitter_archive*

*WeRateDogs* Twitter archive, which includes basic tweet data for over 2,000 tweets that have ratings

In [ ]:
df_twitter_archive

In [ ]:
df_twitter_archive.info()

##### *Columns* 
- *tweet_id*:unique identifier for the Tweet                      
- *in_reply_to_status_id*: Nullable. If the Tweet is a reply, this field will contain the original Tweet’s ID.       
- *in_reply_to_user_id*: Nullable. If the Tweet is a reply, this field will contain the original Tweet’s author ID. This will not necessarily always be the user directly mentioned in the Tweet.        
- *timestamp*: UTC time when this Tweet was created                   
- *source*: Utility used to post the Tweet, as an HTML-formatted string                       
- *text*: The actual UTF-8 text of the status update.                       
- *retweeted_status_id*:           
- *retweeted_status_user_id*:       
- *retweeted_status_timestamp*:     
- *expanded_urls*: Links to the media display page, expanded version                
- *rating_numerator*             
- *rating_denominator*            
- *name*: name of dog, if name is present in text                        
- *doggo*: ('doggo'/'None') based on description in text.                     
- *floofer*: ('floofer'/'None') based on description in text.                      
- *pupper*: ('pupper'/'None') based on description in text.                        
- *puppo*: ('puppo'/'None') based on description in text.                  

In [ ]:
df_twitter_archive.describe()

In [ ]:
sum(df_twitter_archive.duplicated())

In [ ]:
sum(df_twitter_archive['tweet_id'].duplicated())

In [ ]:
# Not all missing 'expanded_urls' are due to retweets and replies
df_twitter_archive[df_twitter_archive['expanded_urls'].isnull()]

In [ ]:
df_twitter_archive['expanded_urls']

In [ ]:
df_twitter_archive[df_twitter_archive['expanded_urls'].str.strip().str[:15]!='https://twitter']['expanded_urls']

In [ ]:
df_twitter_archive['source'].value_counts()

In [ ]:
df_twitter_archive.sample(10)['text'].values

In [ ]:
df_twitter_archive['rating_denominator'].value_counts()

In [ ]:
sum(df_twitter_archive['rating_denominator']>10)

In [ ]:
df_twitter_archive[df_twitter_archive['rating_denominator']>10]['text'].values

In [ ]:
df_twitter_archive['rating_numerator'].value_counts()

In [ ]:
# number of rating_numerator greater than 14 drops off significantly
sum(df_twitter_archive['rating_numerator']>14)

In [ ]:
df_twitter_archive[df_twitter_archive['rating_numerator']>14][['tweet_id','text', 'rating_numerator']].values

In [ ]:
df_twitter_archive['name'].value_counts()

<a id='assess_2'></a>
### *Assess 2. File from Internet: df_images*
Image Prediction file that gives the top 3 predictions of a dog's breed based on a jpeg image associated with each tweet that has been run through a neural network.

In [ ]:
df_images

In [ ]:
df_images.info()

##### *Columns* 
- *tweet_id*: the last part of the tweet URL after "status/" and intersects with the *tweet_id* from `df_twitter_archive`
- *jpg_url*: image url associated with tweet
- *img_num*: image number that corresponds to the most confident prediction (1-4, sincetweets can have up to 4 images)
- *p1*: the algorithm's #1 prediction for the image in the tweet
- *p1_conf*: how confident the algorithm is in its #1 prediction
- *p1_dog*: whether or not the #1 prediction is a breed of dog (TRUE/FALSE) 
- *p2*: the algorithm's second most likely prediction
- *p2_conf*: how confident the algorithm is in its #2 prediction
- *p2_dog*: whether or not the #2 prediction is a breed of dog
- *p3*: the algorithm's third most likely prediction
- *p3_conf*: how confident the algorithm is in its #3 prediction
- *p3_dog*: whether or not the #3 prediction is a breed of dog

In [ ]:
df_images.describe()

In [ ]:
sum(df_images.duplicated())

In [ ]:
sum(df_images['jpg_url'].duplicated())

In [ ]:
# Duplicated jpg_url likely due to retweets
pd.merge(df_images[df_images['jpg_url'].duplicated(keep=False)][['tweet_id','jpg_url']], 
         df_twitter_archive[['tweet_id', 'retweeted_status_id']], 
         how='left', 
         on='tweet_id').sort_values(by='jpg_url')

In [ ]:
df_images['p1'].value_counts()

In [ ]:
df_images['p2'].value_counts()

In [ ]:
df_images['p3'].value_counts()

In [ ]:
pd.concat([df_images['p1'], df_images['p2'], df_images['p3']]).sort_values().unique()

In [ ]:
len(pd.concat([df_images['p1'], df_images['p2'], df_images['p3']]).unique())

In [ ]:
upper_lower = pd.concat([df_images['p1'], df_images['p2'], df_images['p3']]).value_counts()

In [ ]:
lowercase =pd.concat([df_images['p1'], df_images['p2'], df_images['p3']]).str.lower().value_counts()

In [ ]:
(upper_lower - lowercase).sort_values(ascending=True)

In [ ]:
upper_lower['Cardigan'], upper_lower['cardigan']

In [ ]:
df_images[df_images['p2']==('cardigan')]

<a id='assess_3'></a>
### *Assess 3. Twitter API data: df_tweets*
Data gathered from Twitter's API

In [ ]:
df_tweets

In [ ]:
df_tweets.info()

##### *Columns* 
- *tweet_id*: unique identifier for this Tweet.  Intersects with *tweet_id* in `df_twitter_archive`
- *retweet_count*: number of times tweet has been retweeted
- *favourite_count*: approximately how many times the tweet has been liked

In [ ]:
df_tweets.describe()

In [ ]:
max_retweet_id=df_tweets.iloc[df_tweets['retweet_count'].idxmax()]['tweet_id']
max_fav_id = df_tweets.iloc[df_tweets['favourite_count'].idxmax()]['tweet_id']
max_retweet_id, max_fav_id

In [ ]:
min_retweet_id=df_tweets.iloc[df_tweets['retweet_count'].idxmin()]['tweet_id']
min_fav_id = df_tweets.iloc[df_tweets['favourite_count'].idxmin()]['tweet_id']
min_retweet_id, min_fav_id

In [ ]:
df_twitter_archive[df_twitter_archive['tweet_id']==max_retweet_id]

In [ ]:
df_twitter_archive[df_twitter_archive['tweet_id']==min_retweet_id]

In [ ]:
df_twitter_archive[df_twitter_archive['tweet_id']==min_fav_id]

In [ ]:
df_tweets[df_tweets['favourite_count']==0]

In [ ]:
# Discrepancy between 'retweet_count' and'favourite_count'=0 due to tweet_id relates to a retweet
pd.merge(df_tweets[df_tweets['favourite_count']==0], df_twitter_archive[['tweet_id', 'retweeted_status_id']], how='left', on='tweet_id' )

<a id='assess_qual'></a>
### *Assess 4. Quality Issues*
* [Q1.](#qual_1) difference in number of records between the 3 tables: `df_twitter_archive`: 2356, `df_images`: 2075, `df_tweets`: 2337 

`df_twitter_archive`
* [Q2.](#qual_1) 181 *retweeted_status*, 78 *in_reply_to_status* - we only want original tweets
    * [Q2.b.](#qual_2b) redundent columns: *in_reply_to_status_id*, *in_reply_to_user_id*, *retweeted_status_id*, *retweeted_status_user_id*, *retweeted_status_timestamp* 
* [Q3.](#qual_3) Missing data for *expanded_urls*, invalid entries with some entries having multiple urls
* [Q4.](#qual_4) *source* column is difficult to read and contains extra/irrelevent information
* [Q5.](#qual_5) Erroneous datatypes (*tweet_id* integer ==> string, *timestamp* string ==> datetime)
* [Q6.](#qual_6) *rating_denominator* > 10
    * [Q6.b.](#qual_6b)
* [Q7.](#qual_6) Extra large *rating_numerator* (greater than 14)
    * [Q7.b.](#qual_6b)
* [Q8.](#qual_8) Missing *name* and 'a', 'an' and 'the' as names
    * [Q8.b.](#qual_8b) Cooper, Meeko, BOOMER, Atlas, Loki,  Frank, Jack, Oliver

`df_images`
* [Q9.](#qual_5) *tweet_id* should be string
* [Q10.](#tidy_4) Meaningless column headers : *p1*, *p1_conf*, *p1_dog*, *img_num* etc
* [Q11.](#qual_11) 'Cardigan' and 'cardigan' (upper and lower case 'C')



<a id='assess_tidy'></a>
### *Assess 5. Tidiness Issues*
* [T1.](#tidy_1)`df_twitter_archive` table:Stage of dog (*doggo*, *floofer*, *pupper*, *puppo*) one variable so should be one column
    * [T1.b.](#tidy_1b) 10 cases of single tweets with multiple dog stages
* [T2.](#tidy_2) *retweet_count* and *favourite_count* in `df_tweets` table should be part of `df_twitter_archive` table 
* [T3.](#tidy_3)`df_twitter_archive` table: multiple variable in *text* column.  Along with text, it contains a hyperlink, which has the same information as *expanded_url*
* [T4.](#tidy_4) `df_images` table: column headers *p1*,*p2* and *p3* are values,not variable names.  The associated *_conf* and *_dog* columns into 2 columns


<a id='clean'></a>
## Clean

In [ ]:
# Create copies of dataframes for cleaning
df_twitter_archive_clean = df_twitter_archive.copy()
df_images_clean = df_images.copy()
df_tweets_clean = df_tweets.copy()

<a id='clean_miss'></a>
### *Missing and irrelevant data*

<a id='qual_1'></a>
#### `treatments Q1 and Q2:`Remove: retweets, reply tos, tweets without image data in `df_images` and tweets that have since been deleted from server

We only want original ratings (no retweets) that have images.
1. Use the list of *tweet_id* from `df_images` as this has the least number of records (2075 vs 2356 in `df_twitter_archive` and 2337 in `df_tweets`) as base number of records
2. Exclude the tweets that are retweets or replies.  Retweets and replies have a non-null value in *retweeted_status_id* and *in_reply_to_status_id* in `df_twitter_archive`
3. Additionally compare with the *tweet_id* in `df_tweets` as this has less records than `df_twitter_archive` due to deleted records

In [ ]:
# Create master_df that only has the data from df_twitter_archive which also has an image for that tweet_id
master_df=pd.merge(df_images_clean['tweet_id'], df_twitter_archive_clean, how='left', on='tweet_id')
master_df.head()

In [ ]:
# Delete records that are retweets
master_df = master_df[master_df['retweeted_status_id'].isnull()]

In [ ]:
# Delete records that are replies 
master_df = master_df[master_df['in_reply_to_status_id'].isnull()]

In [ ]:
# Delete any records in master_df that don't have a corresponding tweet_id in df_tweets
master_df = pd.merge(df_tweets_clean['tweet_id'],master_df, how='inner', on='tweet_id')

In [ ]:
# Update df_images to exclude any retweets, replies and deleted records
df_images_clean = pd.merge(master_df['tweet_id'], df_images_clean, how='inner', on='tweet_id')


##### Test

In [ ]:
# Should be empty
print(master_df['retweeted_status_id'].value_counts())
print(master_df['in_reply_to_status_id'].value_counts())

In [ ]:
master_df.isnull().any()

In [ ]:
master_df.info()

In [ ]:
df_images_clean.info()

In [ ]:
# Number of records are more than other dataframes but this will be remedied in Tidy section
df_tweets_clean.info()

In [ ]:
# There shouldn't be any duplicated 'jpg_url' after retweets and replies are excluded
sum(df_images_clean['jpg_url'].duplicated())

<a id='qual_2b'></a>
#### `treatments Q2.b`: Redundant columns : *in_reply_to_status_id*, *in_reply_to_user_id*, *retweeted_status_id*, *retweeted_status_user_id*, *retweeted_status_timestamp*

##### Define
Drop the redundant columns: *in_reply_to_status_id*, *in_reply_to_user_id*, *retweeted_status_id*, *retweeted_status_user_id*, *retweeted_status_timestamp*  as they are all populated with NaN

##### Code

In [ ]:
master_df.drop(columns=['in_reply_to_status_id','in_reply_to_user_id',
                       'retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'],
              inplace=True)

##### Test

In [ ]:
master_df.info()

<a id='qual_3'></a>
#### `treatments Q3`: Missing data for *expanded_urls*, invalid entries with some entries having multiple urls

Records that are missing *expanded_urls* have already been excluded due to lack of image prediction in `df_images`

Repopulate *extended_urls* due to some entries containing multiple urls, with some non-twitter urls.
Read `json_tweet.txt` line by line to extract 'expanded_urls' and repopulate *expanded_urls* in `master_df` by creating `df_extended_url` and merging with `master_df`. 

In [ ]:
json_url = []
url_error={}
with open('{}/tweet_json.txt'.format(folder_name)) as f:
    for line in f:
        try: 
            json_data = json.loads(line)          
            tweet_id = json_data['id']
            if ('media') in json_data['entities'].keys():
                url = json_data['entities']['media'][0]['expanded_url']
            else:
                url = json_data['entities']['urls'][0]['expanded_url']
            
            json_url.append({'tweet_id': tweet_id,
                             'expanded_url': url})
           
        
        except Exception as e:
            url_error[json_data['id']]=str(e)

print(len(url_error))

In [ ]:
url_error

In [ ]:
# Should be 0 duplicates
all_tweets = pd.Series(list(master_df['tweet_id']) + list(url_error.keys()))
all_tweets[all_tweets.duplicated()]

#### The *tweet_ids* in url_error do not intersect with *tweet_id* in `master_df`.  Continue with repopulating *expanded_urls* in `master_df`.  If there are later any null entries, the *tweet_ids* in url_error will have to be re-visited.

In [ ]:
df_expanded_urls = pd.DataFrame(json_url, columns=['tweet_id','expanded_url'])

In [ ]:
master_df = pd.merge(master_df, df_expanded_urls, how='left', on='tweet_id')

In [ ]:
# Drop old 'expanded_urls'.  The new column 'expanded_url' also indicates that it is a singular url (drop s from urls)
master_df.drop(columns='expanded_urls', inplace=True)

##### Test

In [ ]:
# Number of records missing 'expanded_url' - should be 0
sum(master_df['expanded_url'].isnull())

In [ ]:
# Check for any 'expanded_url' that is not a twitter WeRateDogs url - should be 0
master_df[master_df['expanded_url'].str[:30]!='https://twitter.com/dog_rates/']

In [ ]:
# Sample check there is only one url in 'expanded_url'
master_df.sample(20)[['tweet_id','expanded_url']].values

In [ ]:
# Check there is only 1 'expanded_url' column
master_df.info()

<a id='clean_tidy'></a>
### *Tidiness*
<a id='tidy_1'></a>
#### `treatments T1`: Stage of dog ('doggo' 'floofer' 'pupper' 'puppo') is one variable split over 4 columns

##### Define

Combine the *doggo*, *floofer*, *pupper* and *puppo* columns into one column named 'dog_stage'.  
Delete redundant columns: *doggo*, *floofer*, *pupper* and *puppo*

##### Code

In [ ]:
master_clean=master_df.copy()
master_clean.head()

In [ ]:
print(master_clean['doggo'].value_counts())
print(master_clean['floofer'].value_counts())
print(master_clean['pupper'].value_counts())
print(master_clean['puppo'].value_counts())

In [ ]:
# Prepare 'doggo', 'floofer','pupper' and 'puppo' columns
# Replace 'None' values with empty string ''
master_clean['doggo'] = master_clean['doggo'].replace('None','', regex=True)
master_clean['floofer'] = master_clean['floofer'].replace('None','', regex=True)
master_clean['pupper'] = master_clean['pupper'].replace('None','', regex=True)
master_clean['puppo'] = master_clean['puppo'].replace('None','', regex=True)

In [ ]:
# Create dog_stage column
master_clean['dog_stage']= (master_clean['doggo'].str.strip()
                            + master_clean['floofer'].str.strip() 
                            + master_clean['pupper'].str.strip() 
                            + master_clean['puppo'].str.strip())


In [ ]:
# Delete columns:'doggo','floofer','pupper','puppo'
master_clean.drop(columns=['doggo','floofer','pupper','puppo'], inplace=True)

##### Test

In [ ]:
master_clean.info()

In [ ]:
master_clean['dog_stage'].value_counts()

<a id='tidy_1b'></a>
#### `treatments T1.b`: 10 cases of single tweets with multiple dog stages


##### Assess

In [ ]:
master_clean[master_clean['dog_stage']=='doggopupper'][['tweet_id','text']].values

In [ ]:
master_clean[master_clean['dog_stage']=='doggofloofer'][['tweet_id','text']].values

In [ ]:

master_clean[master_clean['dog_stage']=='doggopuppo'][['tweet_id','text']].values

##### Define
Individually make the following changes in `master_clean`
1. If *dog_stage* is 'doggopuppo' 
        ==> change to 'puppo' (*tweet_id*:855851453814013952)
2. If *dog_stage* is 'doggofloofer' 
        ==> change to 'doggo' (*tweet_id*:854010172552949760)
3. If *dog_stage* is 'doggopupper' 
        ==> change to 'doggo' (*tweet_id*:785639753186217984)
        ==> change to 'pupper' (*tweet_id*:817777686764523521,801115127852503040)
        ==> change to 'multiple' (*tweet_id*:808106460588765185,759793422261743616,741067306818797568,733109485275860992)
        ==> change to NaN (*tweet_id*:751583847268179968)
        

##### Code


In [ ]:
# list of tweet_id for dogs that need 'dog_stage' changed
dog_list_change = (list(master_clean[master_clean['dog_stage']=='doggopuppo']['tweet_id']) + 
 list(master_clean[master_clean['dog_stage']=='doggofloofer']['tweet_id']) + 
 list(master_clean[master_clean['dog_stage']=='doggopupper']['tweet_id']))
 

In [ ]:
# Change 'doggopuppo' to 'puppo'.  
# Change 'doggofloofer' to 'doggo'
master_clean['dog_stage'].replace('doggopuppo','puppo', inplace=True)
master_clean['dog_stage'].replace('doggofloofer','doggo', inplace=True)

In [ ]:
pupper_list=[817777686764523521,801115127852503040]

In [ ]:
multiple_list=[808106460588765185,759793422261743616,741067306818797568,733109485275860992]

In [ ]:
# change 'doggopupper' to 'doggo' for tweet_id 785639753186217984
master_clean.loc[master_clean['tweet_id']==785639753186217984,'dog_stage']='doggo'

In [ ]:
# change 'doggopupper' to 'pupper' for tweet_id in pupper_list
for pupper in pupper_list:
    master_clean.loc[master_clean['tweet_id']==pupper,'dog_stage'] = 'pupper'

In [ ]:
# change 'doggopupper' to 'multiple' for tweet_id in multiple_list
for id_multi in multiple_list:
    master_clean.loc[master_clean['tweet_id']==id_multi,'dog_stage'] = 'multiple'

In [ ]:
# change 'doggopupper' to '' for tweet_id 751583847268179968
master_clean.loc[master_clean['tweet_id']==751583847268179968,'dog_stage']=np.nan

##### Test

In [ ]:
master_clean['dog_stage'].value_counts()

In [ ]:
master_clean[master_clean['tweet_id'].isin(dog_list_change)][['tweet_id','dog_stage']]

<a id='tidy_2'></a>
#### `treatments T2`: 'retweet_count' and 'favourite_count' should be part of main table, master

##### Define

Merge the *retweet_count* and *favourite_count* columns to the `master_clean` table, joining on *tweet_id*


##### Code

In [ ]:
master_clean = pd.merge(master_clean, df_tweets_clean, how='left', on='tweet_id')

##### Test

In [ ]:
master_clean.head()

<a id='tidy_3'></a>
#### `treatments T3`: *text* column contains hyperlink

##### Define

Delete the hyperlink in the text column as they point to the information in *expanded_url*

Right split *text* on 'https' and keep only the section to the left

##### Code

In [ ]:
master_clean['text'] = master_clean['text'].str.rsplit('https').str[0].str.strip()

##### Test

In [ ]:
master_clean['text'].values

<a id='tidy_4'></a>
#### `treatments T4`: column headers *p1*,*p2* and *p3* in `df_images` are values,not variable names.  The associated *_conf* and *_dog* columns into 2 columns

#### `treatments Q10`: Meaningless column headers : p1, p1_conf, p1_dog, img_num etc


##### Define
Using a for loop, create dictionary of DataFrames `dict_predictions`, with p1, p2 and p3 as the dictionary keys.  

Each dataframe would have the columns *tweet_id*, *prediction_image*, *image_number*, *prediction_number*, *prediction*, *confidence*, *is_dog*.  

*prediction_number* is populated with (1-3), depending on p-number.

Concatenate the 3 dataframes to create `df_image_prediction`


##### Code

In [ ]:
dict_predictions={}

for num in range(1,4):
    p_level = 'p'+str(num)
    dict_predictions[num] = df_images_clean[['tweet_id', 
                                  'jpg_url', 
                                  'img_num',
                                  p_level,
                                  p_level + '_conf',
                                  p_level +'_dog']].rename(columns={p_level:'prediction',
                                                                   (p_level + '_conf'): 'confidence',
                                                                   (p_level +'_dog'): 'is_dog'})
    dict_predictions[num]['prediction_number']=num

    
df_images_predictions=pd.concat(dict_predictions.values())
#df_images_predictions

##### Test

In [ ]:
# (Number of tweet_id in df_images_predictions divided by 3) - (number of tweet_id in df_images-_clean)
# Divided by 3 as there are 3 p-levels
# Should equal 0
len(df_images_predictions['tweet_id'])/3 - len(df_images_clean['tweet_id'])

In [ ]:
df_images_predictions.info()

In [ ]:
# prediction_number should be 1-3
df_images_predictions.describe()

<a id='clean_qual'></a>
### *Quality*
<a id='qual_4'></a>
#### `treatments Q4`: *source* column is difficult to read and has unneccessary information

##### Define
Strip the unnecessary information in the source column to make the data readable and useful, by extracting the string between '>' and '</a'

##### Code

In [ ]:
master_clean['source'].value_counts()

In [ ]:
master_clean['source']= master_clean['source'].str.extract(r'(.+)(>)(.+)(</a>)').iloc[:,2]

##### Test

In [ ]:
master_clean['source'].value_counts()

<a id='qual_5'></a>
#### `treatments Q5 and Q9`: erroneous data types *tweet_id* and *timestamp*

##### Define

Convert *tweet_id* data type to string in `master_df` and `df_images_predictions`.  *tweet_id* is a unique identifier and will not be manipulated with maths. 

Convert *timestamp* data type to datetime


##### Code

In [ ]:
master_clean['tweet_id'] = master_clean['tweet_id'].astype(str).str.strip()
master_clean['timestamp'] = pd.to_datetime(master_clean['timestamp'])

In [ ]:
df_images_predictions['tweet_id'] = df_images_predictions['tweet_id'].astype(str).str.strip()

##### Test

In [ ]:
master_clean.info()

In [ ]:
df_images_predictions.info()

<a id='qual_6'></a>
#### `treatments Q6 and Q7`: *ratings_denominator* > 10 and extra large *ratings_numerator*


#### Define

After excluding retweets and replies, there are two reasons for *ratings_denominator* > 10 and extra large *ratings_numerator*:  
    a) Selecting the incorrect chunk after parsing (including not recognizing decimals in the numerator)  
    b) Multiple dogs
    
<br>
<br>
1. Right Split *text* on '/' and <br>
 
 - select the last number (as a float, to account for decimals) before the '/' for *rating_numerator*  
 - select first number (as integer) after the '/' for the *rating_denominator*
<br>    

2. Recalibrate *rating_numerator* to it's ratio to 10 for any records with *rating_denominator* > 10, to account for multiple dogs.  


#### Code

In [ ]:
# Tally of rating_numerator before cleaning for comparison
master_clean['rating_numerator'].value_counts()

In [ ]:
# Repopulate 'rating_numerator' and 'rating_denominator' based on 'text'
master_clean['rating_numerator']=master_clean['text'].str.rsplit('/',1).str[0].str.extract(r'(\d+\.?\d*)$').astype(float)
master_clean['rating_denominator'] = master_clean['text'].str.rsplit('/',1).str[-1].str.extract(r'^(\d+)').astype(int)

In [ ]:
# Recalibrate 'rating_numerator' to its ratio to 10 for records with 'rating_denominator' greater than 10
master_clean.loc[master_clean['rating_denominator']>10, 'rating_numerator']= (master_clean.loc[master_clean['rating_denominator']>10, 'rating_numerator']
                                                                      /master_clean.loc[master_clean['rating_denominator']>10, 'rating_denominator']
                                                                      *10)

#### Test

In [ ]:
# Compare the 'rating_numerator' with the before snapshot
master_clean['rating_numerator'].value_counts()

In [ ]:
# Check details of records with rating_numerator >14
master_clean[master_clean['rating_numerator']>14][['tweet_id','text','rating_numerator','rating_denominator']].values

####  The records with rating_numerator > 14 will be excluded due to:
1. No rating - 24/7 is not a rating (tweet_id:810984652412424192)
2. Outlier - 1776/10 is a spoof rating (tweet_id:749981277374128128) 
3. Not a dog (tweet_id:670842764863651840)

In [ ]:
# Check rating_denominator for strange values
master_clean['rating_denominator'].value_counts()

In [ ]:
# Check details of records with rating_denominator >10
# rating_numerator should be the exaggerated rating_numerator divided by (rating_denominator times 10)
master_clean[master_clean['rating_denominator']>14][['tweet_id','text','rating_numerator','rating_denominator']].values

<a id='qual_6b'></a>
#### `treatments Q6.b and Q7.b`: Redundant column *rating_denominator*, exaggerated *rating_numerator*, column name *rating_numerator*

##### Define

Following from the treatment repopulating *rating_numerator* and *rating_denominator*
1. Delete *rating_denominator* as all ratings are now out of 10
2. Delete records with *rating_numerator* > 14 
    a) No rating - 24/7 is not a rating (tweet_id:810984652412424192)
    b) Outlier - 1776/10 is a spoof rating (tweet_id:749981277374128128) 
    c) Not a dog (tweet_id:670842764863651840)
3. Rename *rating_numerator* to simply *rating* as there in no longer a denominator

##### Code

In [ ]:
# Drop 'rating_denominator' column
master_clean.drop(columns='rating_denominator', inplace=True)

In [ ]:
# Exclude records with 'rating_numerator' >14
master_clean=master_clean[~(master_clean['rating_numerator']>14)]

In [ ]:
# Rename 'rating_numerator' to 'rating'
master_clean.rename(columns={'rating_numerator':'rating'}, inplace=True)

#### Test

In [ ]:
master_clean.info()

In [ ]:
# Should be empty
master_clean[master_clean['rating']>14]

<a id='qual_8'></a>
#### `treatments Q8`: Missing names and 'a', 'an' and 'the' as names

##### Define
1. Extract names from text using a regex expression finding names after "This is", "Meet", "Say hello to" or "Here is"
2. Delete any non-names (eg "a", "an", "the"
3. For records with missing names, extract names from text using a regex expression finding names after "Named".  This second pass is to account for text that have both "This is" + non-name and "Named" + name


##### Code

In [ ]:
# Extract names from 'text' following "This is", "Meet", "Say hello to" or "Here is"
master_clean['new_name']=master_clean['text'].str.extract(
    r'.*?([Tt]his\sis|[Mm]eet|(?:[Ss]ay\s)[Hh]ello(?:\sto)|[Hh]ere\sis)\s([a-zA-ZÀ-ÿ\']*)[\.,]?\s?.*').iloc[:,1]

In [ ]:
# Confirm list of new_name can be cleaned of non-names by deleting any names starting with a lowercase
master_clean[master_clean['new_name'].str.islower() & ~(master_clean['new_name'].isnull())]['new_name'].unique()

In [ ]:
# Replace all lowercase 'new_name' with NaN
master_clean.loc[master_clean['new_name'].str.islower() & ~(master_clean['new_name'].isnull()),'new_name']=np.nan

In [ ]:
# For all 'new_name'=NaN, Extract name from text, when following "Named"
master_clean.loc[master_clean['new_name'].isna(),'new_name'] = master_clean['text'].str.extract(
    r'.*?([Nn]amed?)\s([a-zA-ZÀ-ÿ\']*)[\.,]?\s?.*').iloc[:,1]

In [ ]:
# Confirm list of new_name can be cleaned of non-names by deleting any names starting with a lowercase
master_clean[master_clean['new_name'].str.islower() & ~(master_clean['new_name'].isnull())]['new_name'].unique()

In [ ]:
# Replace all lowercase 'new_name' with NaN
master_clean.loc[master_clean['new_name'].str.islower() & ~(master_clean['new_name'].isnull()),'new_name']=np.nan

In [ ]:
# Compare 'name' and 'new_name' list for any glaring ommisions
master_clean['new_name'].value_counts() - master_clean['name'].value_counts()

In [ ]:
# Drop 'name' column 
master_clean.drop(columns='name', inplace=True)

In [ ]:
# Rename 'new_name' to 'name'
master_clean.rename(columns={'new_name':'name'}, inplace=True)

##### Test

In [ ]:
master_clean.info()

In [ ]:
# For non NaN 'name', the sum of lowercase 'name' should zero
sum(master_clean[~master_clean['name'].isna()]['name'].str.islower())

<a id='qual_8b'></a>
#### `treatments Q8.b` Cooper, Meeko, BOOMER, Atlas, Loki,  Frank, Jack, Oliver

##### Define
Visual assessment noted 9 names that weren't picked up with regex expression
Select rows where *text* contains 'Cooper', 'Meeko', 'BOOMER', 'Atlas', 'Loki', 'Frank', 'Jack', 'Oliver' and name is NaN and populate *name* with Upper-lower case standard

##### Code

In [ ]:
missing_names = ['Cooper', 'Meeko', 'BOOMER', 'Atlas', 'Loki', 'Frank', 'Jack', 'Oliver']

In [ ]:
for missing_name in missing_names:
    master_clean.loc[(master_clean['text'].str.contains(missing_name
                                                           )) & (
        master_clean['name'].isnull()), 'name'] = missing_name.capitalize()                                      

##### Test

In [ ]:
master_clean[master_clean['text'].str.contains('Cooper')][['tweet_id','text','name']].values

In [ ]:
master_clean[master_clean['text'].str.contains('Meeko')][['tweet_id','text','name']].values

In [ ]:
master_clean[master_clean['text'].str.contains('BOOMER')][['tweet_id','text','name']].values

In [ ]:
master_clean[master_clean['text'].str.contains('Atlas')][['tweet_id','text','name']].values

In [ ]:
master_clean[master_clean['text'].str.contains('Loki')][['tweet_id','text','name']].values

In [ ]:
master_clean[master_clean['text'].str.contains('Frank')][['tweet_id','text','name']].values

In [ ]:
master_clean[master_clean['text'].str.contains('Jack')][['tweet_id','text','name']].values

In [ ]:
master_clean[master_clean['text'].str.contains('Oliver')][['tweet_id','text','name']].values

In [ ]:
master_clean.head()

<a id='qual_11'></a>
#### `treatments Q11.`: Cardigan' and 'cardigan' (upper and lower case 'C')

##### Define
Standardize dog breed predictions so that 'Cardigan' (dog) is 'Cardigan Welsh Corgi' and not mistaken for 'cardigan' (clothing) 

Get list of dog breeds from American Kennel Club Wikipedia page using wptools and store in `df_dog_breeds`.  

The url is: https://en.wikipedia.org/wiki/American_Kennel_Club#Recognized_breeds

Clean list

Map *prediction* in `df_images_predictions` where dog prediction is a dog, ie *is_dog*==True


##### Code

In [ ]:
# Get the page data from Wikipedia 'American Kennel Club'
page_dog_breeds = wptools.page('American_Kennel_Club#Recognized_breeds').get()

In [ ]:
# Store links, that includes the list of dog breeds, in dataframe df_dog_breeds
df_dog_breeds = pd.DataFrame(page_dog_breeds.data['links']).rename(columns={0:'dog_breed'})

In [ ]:
# Create list of keywords to help clean list of non-dog breed entries
list_non_dog=['group','show','kennel','trial','test','title','list','championship','club','breed',
              'United States','Citizen', 'Fifteen and Send Time','Flyball', 'Internationale','International',
              'disorder','dysplasia', 'Incorporation', 'Puppy mill','Lure','Miscellaneous','Pedigree','agility']

In [ ]:
# Iterate through the list of keywords in order to clean df_dog_breeds
for non_dog in list_non_dog:
    df_dog_breeds = df_dog_breeds[~df_dog_breeds['dog_breed'].str.lower().str.contains(non_dog.lower())]

In [ ]:
# Standardize 'dog_breed' column : replace spaces' ' and hyphens'-' with underscores '_'
# replace '_(dog)' with empty string ''
df_dog_breeds['dog_breed']=df_dog_breeds['dog_breed'].str.replace(" ",'_').str.replace("-",'_').str.replace('_\(dog\)','')

In [ ]:
# Function to map and standardize dog breed to 'dog_breed' in df_dog_breeds
def dog_breed_check(check_dog):

    # dictionary of dog breeds that aren't captured in the main function
    dog_dict = {'Brittany_spaniel': 'Brittany_dog',
            'Pekinese': 'Pekingese',
            'Saint_Bernard': 'St._Bernard',
            'Scotch_terrier' : 'Scottish Terrier',
            'Walker_hound' :'Treeing_Walker_Coonhound',
            'wire_haired_fox_terrier' : 'Wire_Fox_Terrier'}
    
    try:
        return (df_dog_breeds[df_dog_breeds['dog_breed'].str.lower()
                              .str.replace('_','').str.contains(
                                  check_dog.lower().replace('_',''))].iloc[0]['dog_breed'])
       
        
    except:
        if check_dog in dog_dict.keys():
            return dog_dict[check_dog]
        else:
            return check_dog # if dog breed is not in df_dog_breeds, do not change 

In [ ]:
# Standardize 'prediction' column.  Replace hyphens '-' with underscore '_'
df_images_predictions['prediction']=df_images_predictions['prediction'].str.replace('-','_')

In [ ]:
#Map 'prediction' column to 'dog_breed' in df_dog_breeds where prediction is a dog, ie 'is_dog'==True
df_images_predictions.loc[df_images_predictions['is_dog']==True,'prediction']=  \
    df_images_predictions.loc[df_images_predictions['is_dog']==True,'prediction'].apply(lambda x: dog_breed_check(x))

##### Test

In [ ]:
df_images_predictions[(df_images_predictions['is_dog']==True) & \
                      (df_images_predictions['prediction_number']==1)]['prediction'].sort_values().unique()

<a id='store'></a>
## Save Cleaned Datasets

In [ ]:
master_clean.to_csv('{}/twitter_archive_master.csv'.format(folder_name), index=False)
df_images_predictions.to_csv('{}/image_predictions.csv'.format(folder_name), index=False)
df_dog_breeds.to_csv('{}/dog_breeds.csv'.format(folder_name), index=False)

<a id='analyze'></a>
## Analyze and Visualize

**WeRateDogs** *has exploded in popularity in the Twittersphere.  Can we pinpoint certain aspects of and patterns in ts Dog Rating Tweets that can help explain why the account has such a huge following?*

Questions that will guide the analysis are:
- [A1](#analyze_1) What is the average dog rating?
- [A2](#analyze_4) Does a higher dog rating correlate with more popularity of the tweet?
- [A3](#analyze_3) How often and when are tweets posted.  Is there a correlation between the day posted and dog rating/popularity of a tweet?
- [A4](#analyze_4) General analysis to address questions such as
    - What are the most common dog names?
    - Do certain names tend to get a higher rating?  Tend to be more popular?
    - Do tweets with a dog name tend to be more popular than tweets without a dog name?
    - How does a dog's stage relate to a tweet's popularity
- [A5](#analyze_5) Are more humourous tweets more popular?

<a id='analyze_1'></a>
#### Distribution of Dog Ratings

In [ ]:
# Title and axis format
title_font = {'fontname':'Arial', 'size':'16', 'color':'black', 'weight':'normal',
  'verticalalignment':'bottom'} # Bottom vertical alignment for more space
axis_font = {'fontname':'Arial', 'size':'14'}

In [ ]:
plt.figure(figsize=(7,7))
sns.distplot(master_clean['rating'], kde=False)
plt.title('Distribution of Dog Ratings', fontdict=title_font)
plt.ylabel('Number of Ratings', fontdict=axis_font)
plt.xlabel('Rating', fontdict=axis_font)
plt.annotate(('Lower Quartile: {}\nMedian: {}\nUpper Quartile: {}'.format(
    master_clean['rating'].describe()['25%'],
    master_clean['rating'].describe()['50%'],
    master_clean['rating'].describe()['75%'])),
    xy=(0.05, 0.50), xycoords='axes fraction', fontname='Arial', size='14',
             bbox=dict(facecolor='none', edgecolor='black', boxstyle='square, pad=0.7'));

##### Conclusion
The Dog Ratings system in unique in that it generally hands out scores greater than 10/10.  Scores are heavily left skewed and tend to the median of 11, maxing out at 14.

#### Definition of Popularity
We can proxy a tweet's popularity by how many retweets and likes (favourites) it has, with a higher number in both equating to more popular.  Patterns relating to the popularity of the dog rating tweets can help explain the success of *WeRateDogs*

In [ ]:
# Plot correlation between Retweets and Likes
plt.figure(figsize=(7,7))
sns.scatterplot(x='retweet_count', y='favourite_count', data=master_clean)
plt.title('Correlation between number of Retweets and number of Likes', fontdict=title_font)
plt.xlabel('Number of Retweets', fontdict=axis_font)
plt.ylabel('Number of Likes',fontdict=axis_font)
plt.annotate('Correlation (p-value) = {}'.format(
    round(master_clean['retweet_count'].corr(master_clean['favourite_count'], method='pearson'), ndigits=4)),
    xy=(0.05, 0.90), xycoords='axes fraction', fontname='Arial', size='14',
             bbox=dict(facecolor='none', edgecolor='black', boxstyle='square, pad=0.3'));

##### Conclusion
Number of retweets and number of likes are highly positively correlated with each other, with a correlation coefficient of 0.9296.  Thus any further analysis involving popularity will just use *Number of Retweets* as proxy.  

Furthermore, this aligns with our investigation of what contributes to the popularity and high following of *WeRateDogs*. While likes and favourites reflect the quality of engagement with followers, Retweets does this plus further distributes a tweet, reaching out to potential followers.

<a id='analyze_3'></a>
#### Timing of Tweet Posts
Does the timing of when a tweet is posted correlate with how popular the tweet becomes?

In [ ]:
# Create 'hour' 'day_of_week' 'month' columns in master_clean based on 'timestamp'
master_clean['hour']= master_clean['timestamp'].apply(lambda time: time.hour)
master_clean['day_of_week']= master_clean['timestamp'].apply(lambda time: time.dayofweek)
master_clean['month']= master_clean['timestamp'].apply(lambda time: time.month)
master_clean['year']= master_clean['timestamp'].apply(lambda time: time.year)

In [ ]:
master_clean.head()

In [ ]:
# map day of week integer to actual string name
dmap = {0:'Mon', 1:'Tue', 2:'Wed',3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
master_clean['day_of_week']= master_clean['day_of_week'].map(dmap)

In [ ]:
master_clean.head()

In [ ]:
order_days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

In [ ]:
# Number of tweets broken down by day of the week  per month
day_month_count = master_clean.groupby(by=['day_of_week','month']).count()['tweet_id'].unstack()\
.sort_index(axis=1, ascending=True).reindex(order_days, axis=0)
day_month_count

In [ ]:
# Number of retweets broken down by day of the week per month
day_month_retweet = master_clean.groupby(by=['day_of_week','month']).sum()['retweet_count'].unstack()\
.sort_index(axis=1, ascending=True).reindex(order_days, axis=0)
day_month_retweet

#### `Timing 1` How have number of tweets and retweets compared over time?

A comparison of Tweet posting and Retweet activity, to discover whether the number of tweet posted correlate with the number of retweets.

In [ ]:
master_clean.groupby(master_clean['timestamp'].dt.to_period('m'))['tweet_id'].count()

In [ ]:
master_clean.groupby(by=(master_clean['timestamp'].dt.to_period('m')))['retweet_count'].sum()

In [ ]:
fig,ax = plt.subplots()


#plot number of tweets over time
master_clean.groupby(by=(master_clean['timestamp'].dt.to_period('m')))['tweet_id'].count().plot(colormap='Set1', 
                                                                                             figsize=(8,5))
#set second y-axis
ax2 = ax.twinx()

# plot number of retweets over time
master_clean.groupby(by=(master_clean['timestamp'].dt.to_period('m')))['retweet_count'].sum().plot(colormap='tab10',
                                                                                              figsize=(8,5))
# Set title and axis names
plt.title('Number of Tweets (Left Axis) and Retweets (Right Axis) over Time', fontdict=title_font)
ax.set_ylabel('Number of Tweets', fontdict=axis_font)
ax2.set_ylabel('Number of Retweets',fontdict=axis_font, 
               rotation=270, va='bottom')
ax.set_xlabel('Date',fontdict=axis_font)

# Label graph
fig.text(x=0.73, y=0.19, s='Tweets\n(left axis)', fontdict=axis_font,
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='square, pad=0.3'))
fig.text(x=0.73, y=0.5, s='Retweets\n(right axis)', fontdict=axis_font,
         bbox=dict(facecolor='white', edgecolor='black', boxstyle='square, pad=0.3'))

plt.tight_layout();


#### `Timing 2` What is the distribution over a year of when Tweets are posted and when Popular Tweets are posted

Compare the pattern of the distribution over a the year and broken down by day of the week of
- Tweet Postings
- Popular Tweet postings.

Tweet postings identifies any pattern in posting activity over the course of a year broken down by day of the week

Popular Tweet Postings identifes any pattern of when popular tweets are posted, by aggreagating the number of retweets, broken down by month and day of the week. 

In [ ]:
# Heatmap distribution of how popular tweets are based on when they are posted, 
# taking into account pattern on tweet posting
figure,axes = plt.subplots(nrows=2, ncols=1, figsize=(14,14))

sns.heatmap(day_month_count, cmap='coolwarm',linecolor='white', linewidth=3, ax=axes[0])
axes[0].set_title('Distribution of When Tweets are Posted Over a Year', fontdict=title_font)
axes[0].set_ylabel('Day of the Week')
axes[0].set_xlabel('Month')

sns.heatmap(day_month_retweet, cmap='coolwarm', linecolor='white', linewidth=3, ax=axes[1])
axes[1].set_title('Distribution of Popular Tweet Postings Over a Year', fontdict=title_font)
axes[1].set_ylabel('Day of the Week')
axes[1].set_xlabel('Month')

plt.tight_layout;


##### Conclusion
Although the number of Tweets have dropped over time, the Retweet activity has remained elevated after a dip, and now trending over 2000 retweets per month.  The drop at the end is due to the dataset ending with an incomplete month of data. 

Interestingly, retweets were quite elevated towards the inception of the account, even though one would assume less followers.  This suggests that either a few Tweets were very popular early on or followers that jumped on board later on re-deiscovered and retweeted these early Tweets.  I think combined with the increased Tweeting activty early on both of these factors were at play and that these early Tweets were the impetus to the increasing popularity of *WeRateDogs*  

More tweets are posted in the months of November and December.  However this does not coincide with when the most popular tweets are posted.  While tweets tended to be more popular if they were posted in December, the most popular tweet and/or tweets are posted on a Saturday in June.  

Since the heatmap is an aggregation of retweets for each month and day of week, the hotspot in June could be due to posting of extremely popular posts that were retweeted extensively, or due to increased Tweet posting activity with a larger number of Tweets retweeted at a moderate level.  However, in conjunction with the heatmap showing the distribution of Tweet posting activity, the former reason is more feasible, that a few extremely popular post were retweeted extensively.



<a id='analyze_4'></a>
#### Analysing the most popular tweets

Ordering the data based on popularity (*retweet_count*), common themes can be found as to what makes a tweet popular and thus contributing to the popularity and following of *WeRateDogs* 

In [ ]:
# Top 20 Most common dog names
master_clean['name'].value_counts().head(20)

In [ ]:
# Create a dataframe merging element of master and image_predictions 
master_image_pred_1=pd.merge(master_clean[['tweet_id','timestamp','month','day_of_week',
                                           'rating','retweet_count','name', 'dog_stage']],
        df_images_predictions[df_images_predictions['prediction_number']==1][['tweet_id', 
                                                                              'prediction','confidence','is_dog']],
        on='tweet_id', how='inner')

In [ ]:
master_image_pred_1['retweet_count'].describe()

In [ ]:
master_image_pred_1['timestamp'].min(), master_image_pred_1['timestamp'].max()

In [ ]:
master_image_pred_1.sort_values(by='retweet_count', ascending=False).head(20)

In [ ]:
master_clean[['tweet_id','timestamp','text','retweet_count', 'expanded_url']]\
                    .sort_values(by='retweet_count',ascending=False).head(10).values

In [ ]:
master_clean[master_clean['name']=='George']

In [ ]:
master_image_pred_1.sort_values(by='retweet_count', ascending=True).head(10)

##### How are dog stages represented?

In [ ]:
#Pie chart
labels_dog_stage= list(master_clean['dog_stage'].value_counts().index)[1:]
sizes_dog_stage = list(master_clean['dog_stage'].value_counts().values)[1:]
labels_has_stage= ('No Dog Stage','')
sizes_has_stage=list(master_clean['dog_stage'].value_counts().values)[0],sum(sizes_dog_stage)
#colours
colours_dog_stage=['#4a63d4','#b40426','#92b4fe','#d6dce4', '#ef896c']
colours_has_stage = ['#DCDCDC','#f7b295']

# Explode slice 'doggo' and 'Has Dog Stage'
explode_dog_stage = (0,0.2,0,0,0)
explode_has_stage=(0,0.2)

In [ ]:
fig = plt.figure(figsize=(10,8))


axes1 = fig.add_axes([0.1, 0.1, 0.3, 0.3]) 
axes2 = fig.add_axes([0.4, 0.3, 0.6,0.6])

axes1.pie(x=sizes_has_stage, explode=explode_has_stage,
          colors=colours_has_stage, labels=labels_has_stage, 
          startangle=70,shadow=True)
axes1.set_title('Distribution of\nNo Dog Stage vs. Has Dog Stage')

axes2.pie(x=sizes_dog_stage, explode=explode_dog_stage,
          colors=colours_dog_stage, labels=labels_dog_stage, 
          shadow=True)
axes2.set_title('Representation of Dog Stages' , fontsize=20)




bbox_props = dict(boxstyle="rarrow,pad=0.3", fc="white", ec="black", lw=2)
t = fig.text(0.4, 0.38, "Has Dog Stage", ha="center", va="center", rotation=30,
            size=18,
            bbox=bbox_props)


##### Conclusions

- 3 out of 5 of the most popular tweets, based on retweets, were posted in June, with the top 2 both posted on a Saturday in June, 3 weeks apart.  In contrast the least popular 10 tweets were all posted in November 2015.  The least popular tweets occurred at the beginning of the dataset, when the *WeRateDogs* Twitter account was just beginning so understandably would not have many followers to retweet.  

- Of note, in the top 10 popular tweets are tweets from early on in the dataset, on 14th and 20th December 2015.  According to image prediction , these are considered 'humourous' as their image predictions resulted in bubble and swing.  Perhaps with the popularity of these tweets began the rapid rise in popularity and followers of *WeRateDogs*.  It is interesting that around the time of these early popular tweets in December 2015, *WeRateDogs* significantly decreased the number of dog rating Tweets, which started at around 300 per month, peaking at about 350 in December 2015 and dropping to around 50 per month from April 2016, a level maintained since.  

- Top 12 most popular tweets account for the top quartile for number of retweets, with a range occuring in the years 2015-2017, thus length of time since posting may not neccessarily be such a strong factor contributing to the large number of retweets.

- Half the dogs in the most popular 10 tweets have a name, with neither of the top 2 Tweets having mention of the dog name.  Futhermore, only 1 of the 9 names in the 20 top popular posts are found in the top 20 most common dog names.  That name is Buddy, which is common dog name number 20. 

- 3 of the dogs in the top 10 are classified at the *doggo* dog stage.  In fact the number 1 and 2 popular Tweets are about *doggos*.  This is interesting, considering that doggos represent a small percentage of the total tweets, and if we exclude tweets without a dog stage mentioned, *doggos* still only represent about a quarter.

- The top 20 most popular Tweets have ratings above 10, with 16 of the Tweets having ratings in the upper quartile.  In contrast the 10 least popular Tweets have ratings 11 and below, with 6 having a rating in the lower quartile.  Again these least popular tweets were in the infancy of the account and most likely before the workings of the generous rating system were ironed out

- The top 3 most popular Tweets included a video and according to the text, the dog is doing something that is not usual for a dog
    - Here's a doggo realizing you can stand in a pool.
    - Here's a doggo blowing bubbles.
    - This is Stephan. He just wants to help.

<a id='analyze_5'></a>
#### Is there a relationship between humourous tweets and dog rating or popularity?

*WeRateDogs* is known for its humourous Tweets.  Perhaps humourous tweets are more popular and readily shared, thus increasing engagement and followers.

`Humour 1`If we first define a humourous tweet as one where the dog may not be so easily visible in the photo due to perhaps blending in to the background or in costume, we could use the image prediction data.  A prediction that is not a dog could be a proxy for humour.  We will use the data from the most confident prediction

In [ ]:
# Plot the relationship between ratings and retweet_count, 
#separating for whether the most confident image prediction was a dog or not

#plt.figure(figsize=(12,12))

g=sns.lmplot(x='rating',y='retweet_count',data=master_image_pred_1,col='is_dog', fit_reg=False,
           scatter_kws={'s':100},palette='coolwarm', height=5, aspect=1, legend=False)
plt.subplots_adjust(top=0.7)
g.fig.suptitle('Popularity vs. Dog Rating: Comparing Distribution of Humorous Tweets\nBased on Image Prediction', 
               fontdict=title_font, fontsize='16')
g.set_xlabels('Dog Rating', fontdict=axis_font)
g.set_ylabels('Popularity (Number of Retweets)', fontdict=axis_font)

g.fig.axes[0].set_title('Humorous Tweet\nImage Prediction: Not a Dog')
g.fig.axes[1].set_title('Non-Humorous Tweet\nImage Prediction: A Dog')

plt.tight_layout;

`Humour 2`  Alternatively, we could search the tweet text for common 'joke' phrases, suggesting there is no dog in the picture.  

Common joke phrases are: 
- We only rate dogs
- Please only send photos of dogs
- Another photo without a dog

In [ ]:
# Select rows in master_clean that contain joke phrases in 'text' and set 'humourous_comment' to True
master_clean.loc[master_clean['text'].str.lower().str.contains('we only rate dogs') | \
                 master_clean['text'].str.lower().str.contains('please only send') | \
                 master_clean['text'].str.lower().str.contains('without a dog'),'humourous_comment']=True


In [ ]:
# 'humourous_comment' set to False if text does not contain joke phrase
master_clean.loc[master_clean['humourous_comment'].isnull(),'humourous_comment']=False

In [ ]:
master_clean.head()

In [ ]:
# Plot the relationship between ratings and retweet_count, 
#separating for whether the text contained a joke phrase
g=sns.lmplot(x='rating',y='retweet_count',data=master_clean,hue='humourous_comment', fit_reg=False,
           markers=['o','v'],scatter_kws={'s':200},
             palette='coolwarm', height=5, aspect=2, legend=False, legend_out=True)
plt.subplots_adjust(top=0.8)
g.fig.suptitle('Popularity vs. Dog Rating: Comparing Distribution of Humorous Tweets\n(Based on Text)', 
               fontdict=title_font, fontsize='16')
g.set_xlabels('Dog Rating', fontdict=axis_font)
g.set_ylabels('Popularity (Number of Retweets)', fontdict=axis_font)

plt.legend(title='Humorous Tweet', loc='upper left')
plt.tight_layout;

##### Conclusion
Comparing Popularity of a Tweet with the Dog Rating given, there does appear that higher dog rating Tweets are more popular.  However we must take into consideration that over 75% of the Tweets have a dog rating greater than 10, thus increasing the possibility that tweets with higher ratings become more popular.

When using either Image Prediction or Text as proxy for humour, the humourous Tweets are not the the most popular and otherwise trend similarly to non-humourous Tweets. 

This is however an imperfect system to define humourous Tweets.   

<a id='conclusion'></a>
## Conclusions
- Dog Ratings, while generous and routinely above 10/10, still trend around the median of 11 and max out at 14 
- Tweeting activity giving out Dog Ratings has decreased significantly since inception of the *WeRate ogs* account, while Retweets have remained elevated.  The early Tweets were retweeted extensively and possibly were the impetus for the huge following of the *WeRateDogs* account to the point where retweet activity is sustained even though Tweeting dog ratings has decresed.
- Tweeting activity is generally higher in November and December with tweets posted in December and January more popular.  However tweets posted on Saturday in June were retweeted the most.  This can be attributed to the two most popular Tweets.
- The top 12 Tweets, out of nearly 2000, account for the top quartile of popularity (number of retweets).  These tweets span across the years 2015-2017
- 5 of the dogs in the 10 most Popular Tweets had their name mentioned.  Only one of the 9 names mentioned in the 20 most popular Tweeets were in the top 20 common names in the dataset
- *doggo* dog stage was disproportionately represented in the top 10 Tweets, 3 out 10, with 2 of them the 2 most popular tweets.
- Humourous Tweets aren't generally more popular than non-humours tweets.  This could be due to an imperfect system in defining humourous tweets among a set of Tweets that are known to be humourous.
- The top 3 popular Tweets included a video showing activities not normally done by a dog - standing in a pool, blowing bubbles, just trying to help.


### Extensions

##### Data Wrangling
Data wrangling of the *WeRateDog* Twitter data uncovered both *Quality* and *Tidiness* issues.  A bulk of the assessment and cleaning involved reading the Twitter text.  Thus being able somehow to assess programatically as opposed to visually may be more time efficient and less prone to human error.

##### Analysis
Other interesting avenues of analysis could include:
* How have the Dog Ratings trended over time?
* How has the number of followers trended over time?
    - Were there spikes in number of followers that coincide a Tweet?
* Are there certain key words in the text of the most popular Dog Rating Tweets?
* The timing of retweets from retweet date as opposed to date of original post
* Are Tweets with a video included more popular?   





### Resources### 

[6 signs of 
dirty data](https://www.umbel.com/blog/audience-data/6-signs-dirty-data-infographic/)

[check encoding of file](https://stackoverflow.com/questions/37177069/how-to-check-encoding-of-a-csv-file)

https://stackoverflow.com/questions/28384588/twitter-api-get-tweets-with-specific-id

https://knowledge.udacity.com/questions/9781

[json.dump each variable on new line](https://stackoverflow.com/questions/17055117/python-json-dump-append-to-txt-with-each-variable-on-new-line)

https://stackoverflow.com/questions/21308762/avoid-twitter-api-limitation-with-tweepy

https://stackoverflow.com/questions/18869688/twitter-api-check-if-a-tweet-is-a-retweet

https://knowledge.udacity.com/questions/33009

https://stackoverflow.com/questions/49161120/pandas-python-set-value-of-one-column-based-on-value-in-another-column

[sort weekday on heatmap](https://stackoverflow.com/questions/47741400/pandas-dataframe-group-and-sort-by-weekday?rq=1)

[create dataframes in a loop](https://stackoverflow.com/questions/55277587/create-dataframe-in-a-loop)

[Dog breed list](https://en.wikipedia.org/wiki/American_Kennel_Club#Recognized_breeds)

[Implot Title](https://mlwhiz.com/blog/2015/09/13/seaborn_visualizations/)